
# Upper Irwell Investment tool

Welcome to the Upper Irwell Investment tool. This workbook will take you step by step through the process of identifying priority areas of investment in NFM schemes based on existing data that the EA holds.


# 1. Introduction
This workbook is a unified workflow that combines the data and functionalities of four existing EA owned tools:

1.	NFM Opportunities Map (by JBA Consultants): Identifies areas capable of storing different water volumes and lists potential features for flood mitigation.
2.	NFM Storage Tool (by Jacobs): Analyzes how varying storage volumes affect flood hydrographs, return periods, and properties at risk.
3.	EA Property reports for all the communities at risk from the NW Opportunities Map/Toolkit (Jacobs) identifies numbers of properties at risk by risk band and deprivation, and weighted annual average damages. Also has lots of natural capital information. This is a GMMC only held data set.
4.	DEFRA Funding Calculator: Calculates potential funding released by DEFRA for flood mitigation schemes based on changes in property risk levels.

It is important to note that this workbook is solely trying to make the best use of exisitng data and information. It should only be used as a scoping tool rather than as a detailed planning tool. Many assumptions and approximations are made throughout this workbook to be able to make this workflow. These are highlighted throughout the workbook.

## How to use this workbook
The Upper Irwell Investment tool consists of python code presented in a jupyter workbook, and a folder of data files associated with it. Rather than hiding the code away behind a user interface, it is presented here so that tool can be easily developed in the future.

This is the master copy of the code and no one can edit it, so don't worry about making mistakes. To edit this document you must click file -> save a copy in drive and then edit that document instead. (you will need a google account to do this)

### Uploading the data
Go to the sharepoint site and download the folder 'Irwell_Tool_Data'. This contains all of the files you need to run the workbook. If this has downloaded as a zip file, make sure you unzip it to be able to access all of the files inside. Now go back to the google workbook and click on the folder symbol to the left of this text in colab. A sidebar should appear. Either drag and drop all of the files into the sidebar, or click the upload file symbol at the top and navigate to the files to upload. Some of the files are quite large, so make sure they have all uploaded properly before you run the workbook. You can see the progress of the uploads with a loading circle in the sidebar. Colab does not save this data, so you will need to upload it every session. Remeber to upload data to the main folder NOT the sample data folder.

### Downloading the Results
Results are stored in a folder called 'Results'. Make sure that you download files and save them to you own computer as they will be deleted from the colab storage when you close the workbook (or if you let the session expire). Navigate to the files you want to download then click the 3 dots next to the file and selevt download. Find the files you downloaded and rename/move them in accordance with your own naming conventions and file system. I have included the time and date that the file was created in the file name so that none of your results are overwritten. You need to rename them so they make sense to you!

### What is python code?
Do not be intimidated by the code!! Coding is just writing instructions in the correct language (python) and instructing the installed interpreter (python) to run the program on your computer. Rather than having to install python onto your computer, we will be working in google colab to avoid any intallation issues. Colab is basically letting us run python code on their servers in the cloud rather than running python on our own machine. This is why we need to upload any data that we want python to look at to the cloud.

We are working in what is known as a 'Jupyter notebook' which lets us develop our code in a very interactive way, allowing us to have text, images, figures and code all in one place. You will see alternating text blocks and code blocks throughout this notebook. When code blocks are executed, graphs, text or data tables may be outputted beneath them.

Ultimately, we are just writing instructions and getting a computer to interpret them. There are very few lines of code that you need to change in this workbook and they will be clearly marked with comments in the code. A python comment looks like this:



```
# This is what a comment looks like. Anything that follows a hash symbol is ignored by python, so we can write human readable instructions directly into code blocks.
```

### Running the code
Once you have made your changes to the code and have uploaded the associated dataset, you can execute the whole workbook by going to 'Runtime -> Run all' at the top of the page. This will run all of the code cells in the workbook. Note that 'optimisation' cells towards the end of the notebook may take a very long time (hours) to run. You may therefore wish to execute single cells of code instead. You can do so by pressing the 'play' button at the top left of a code cell.

Note: Python works by running instructions in order. In the code we create data objects and variables that are held in the computer's memory. If we run a code cell again with different values, the variables will be overwritten in the memory of the computer with these new values. So if you start running cells out of order, then you may get some confusing results or some errors.

### Errors
When python can't understand something, it will create an error message beneath the code cell. These errors can be quite hard to interpret, even if you are familiar with python. Given how few lines of code there are to edit, it is unlikely that you will get any errors. The most likely ones that you will come across will because:
- The data files have not uploaded correctly. Some of the files are big and if there is any disruption to the upload then they may be only partially there. Try again!
- Typos. Code is VERY strict about characters, spaces, capitilization and punctuation. Check that everything is typed correctly then try again!
- Searching for something that doesn't exist. It may be that a catchment or community that we thought was captured in the data isn't. You can check this by looking in the uploaded data files.

There is an AI function in colab that will help you understand the errors better. Alternatively, copy and paste your error into google and look for similar problems on Stack Overflow. This is exactly what professional programmers do!

If you feel like you've broken the code beyond repair, you can always start over and make a new copy of this master version! Or email elizabeth.lewis-3@manchester.ac.uk

### Learning to code
If you would like to learn how to code from the very beginning I have given you access to my 'introduction to python' training workbooks for free here:

https://colab.research.google.com/drive/1CYsvSjWnFX8pzZDxxKw71VLCZAEicMgY?usp=sharing

https://colab.research.google.com/drive/1KPeFL5e6A5i27c2eBqim4nKb7i5nT4Lc?usp=sharing



### Workbook structure

1.   Introduction
2.   Importing functionality
3.   Reading in the NFM opportunities map data
4.   Reading in the JACOBS storage tool data
5.   Functionality for calculating changes to communities at risk
6.   Calculating changes to an individual community
7.   Calculating funding information for the whole of the Upper Irwell



# 2. Importing functionality

The code in this section imports additional python packages to help us work with map data, tables of data, produce interactive graphs and more.

We also have our own bespoke module 'calculator.py' this is the DEFRA funding calculator rewritten as python code. As the code is quite lengthy, it written as it's own python package that we import into the workbook, rather than being shown in its entirety here. The calculator code can be easily edited if needed in a text editor such as notepad.

None of the code will run without this cell being executed first. It only needs to be executed once per session and may take a few moments to run.

In [0]:
%pip install folium openpyxl geopandas

In [0]:
dbutils.library.restartPython()

In [0]:
import os
import sys
from datetime import datetime

# working with numerical data
import pandas as pd
import numpy as np
import pickle
import openpyxl

# working with spatial data
import geopandas as gpd
from shapely.geometry import Point

# Making graphs
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import folium
import branca.colormap as cm
import matplotlib.cm as mpl_cm
import matplotlib.colors as mpl_colors


In [0]:
# Get the current user 
user =  dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()

# Create the user workspace path for the calculator module
sys.path.append(f"/Workspace/Users/{user}/EA-upper-irwell-tool")

import calculator

# Setting source and output paths

In [0]:
data_path = "/dbfs/mnt/lab/unrestricted/irwell_data"

results_path = f"/Workspace/Users/{user}/EA-upper-irwell-tool/results"

if not os.path.exists(results_path):
    os.makedirs(results_path)

# 3. Reading in the NFM opportunities map data

The JBA Natural Flood Management (NFM) maps visualise the changes that NFM measures are predicted to have on surface water runoff, in terms of peak surface runoff reduction and timing of the peak runoff.

Descriptions of the maps are found here: https://assets.publishing.service.gov.uk/media/6036c659d3bf7f0ab2f070c1/Working_with_natural_processes_mapping_technical_report.pdf It details how these numbers have been calculated.

## Map attributes

NFM features have been aggregated to the water body scale, which we use here from the file 'a0000002b.gdbtable'. The water body map contains the following attributes:

- 'WB_NAME': Water body name
- 'Headwater': River name
- 'EA_WB_ID': Water body ID- EA assigned
- 'JBA_CAT_ID': Water body ID- JBA assigned
- 'Extra_Pond_Capacity_RP100_10': Additional potential storage based on a 100yr flood
- 'Existing_Pond_Volume_RP100_10': Existing storage based on a 100yr flood
- 'Area_km2': Water body area
- 'Workshop_WfW_Area_km2': Woodlands for Water area, refined at stakeholder workshop
- 'Workshop_ImpGrass_Area_km2': Improved grassland area, refined at stakeholder workshop
- 'Workshop_Urban_Area_km2': Urban extent
- 'Workshop_CombLoss_Area_km2': Area of improved soil structure, resulting in enhanced soil moisture storage capacity in rural areas and increased green spaces in urban areas
- 'Losses_PercRed_RP30': Percent reduction in the 30yr flow due to enhanced rural and urban losses
- 'WfW_PercRed_RP30': Percent reduction in the 30yr flow due to increased woodlands
- 'RAF_PercRed_RP30': Percent reduction in the 30yr flow due to runoff attenuation features
- 'Losses_PercRed_RP100: Percent reduction in the 100yr flow due to enhanced rural and urban losses
- 'WfW_PercRed_RP100': Percent reduction in the 100yr flow due to increased woodlands
- 'RAF_PercRed_RP100': Percent reduction in the 100yr flow due to runoff attenuation features
- 'Base_ToP_RP30': Time of peak flow in the 30yr design event
- 'Losses_ToP_RP30': Time of peak flow in the 30yr design event under enhanced rural and urban losses scenario
- 'WfW_ToP_RP30': Time of peak flow in the 30yr design event under enahnced woodland scenario
- 'RAF_ToP_RP30': Time of peak flow in the 30yr design event under enhanced runoff attenuation feature scenario
- 'Base_ToP_RP100': Time of peak flow in the 100yr design event
- 'Losses_ToP_RP100': Time of peak flow in the 100yr design event under enhanced rural and urban losses scenario
- 'WfW_ToP_RP100': Time of peak flow in the 30yr design event under enahnced woodland scenario
- 'RAF_ToP_RP100: Time of peak flow in the 30yr design event under enhanced runoff attenuation feature scenario

## How can we use this data?

Only the Runoff Attenuation Feature layer reports volumes of water stored. To use the other layers, we will need to approximate increased storage of water from increased area or improved soil/woodland:
- Assuming a soil depth of 1m and an increase in 10% storage capacity of the soil, we can calculate a volume of water reatined by the NFM features.
- 1km2 = 1000000m2,
- with a depth of 1m, 1km2 = 1000000 m3 of soil
- with an increase of 10% storage, 1km2 = 100000 m3 of water stored

## Assumptions relating to the use of the NFM maps
- Potential storage capacity is distributed evenly across the water body
- Values relating to the 100yr simulations are reflective of the maximum storage capacity and are applicable at all return periods.
- Equivalent storage can be approximated from land cover change scenarios
- Storage from different NFM features can be stacked (i.e. added together to find a maximum potential storage)

## Running the code

The first line of code in this section can be changed to only include NFM types of interest. The relevant NFM features we can investigate are:

- 'Extra_Pond_Capacity_RP100_10' Adding pond capacity
- 'WfW' adding woodland
- 'Losses' adding extra storage in rural and urban soil
- 'ImpGrass' adding extra storage trough improving the storage capacity of soil in grassland

If you want to explore all options combined use the following line of code:

```
NFM_features_to_include = ['Extra_Pond_Capacity_RP100_10', 'WfW', 'Losses', 'ImpGrass']
```

If you want to just look at using only certain NFM features, delete the names that are not of interest. For example, if you are only interested in adding ponds use the following line of code:

```
NFM_features_to_include = ['Extra_Pond_Capacity_RP100_10']
```

Or if you want to look at ponds and improved grassland use the following:

```
NFM_features_to_include = ['Extra_Pond_Capacity_RP100_10', 'ImpGrass']
```
Note that every aspect of punctuation is important. If you are missing a ' or , you will get an error!

Nothing else in this section should be changed. Similar to section 1, the code in this section only needs to be run once per session to read in the NFM opportunities data. Reading data into memory is relatively slow, so it's better to avoid reading everything in again if you don't have to. If you change the NFM combination to explore, these code blocks should be executed again.

In [0]:
# Select which NFM features to consider
# The line of code below can be changed. Delete layers as applicable from the list ["Extra_Pond_Capacity_RP100_10", "WfW", "Losses", "ImpGrass"] below
NFM_features_to_include = ["Extra_Pond_Capacity_RP100_10", "WfW", "Losses", "ImpGrass"]

# Read in the NFM spatial data
NFM_features_gdf = gpd.read_file(f"{data_path}/a0000002b.gdbtable")
NFM_features_gdf["WfW"] = NFM_features_gdf["Workshop_WfW_Area_km2"]*10000
NFM_features_gdf["Losses"] = NFM_features_gdf["Workshop_CombLoss_Area_km2"]*10000
NFM_features_gdf["ImpGrass"] = NFM_features_gdf["Workshop_ImpGrass_Area_km2"]*10000
NFM_features_gdf["total_NFM_storage"] = NFM_features_gdf[NFM_features_to_include].sum(axis=1)

In [0]:
# Read in the WFD water bodies shapefile
wb_gdf = gpd.read_file(f"{data_path}/WFD_Cycle_2_River_Water_Body_Catchments.shp")

In [0]:
# Plot the NFM storage data and the WFD boundaries
ax = NFM_features_gdf.plot(column="total_NFM_storage", cmap="viridis", legend=True, legend_kwds={"label":"Total NFM storage (cubic m)"})
wb_gdf.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=1, legend=True, legend_kwds={"label":"WFD water bodies", "loc": "upper left"})

# Define a custom legend entry
legend_elements = [Line2D([0], [0], color="black", lw=1, label="WFD water bodies")]
ax.legend(handles=legend_elements, loc="upper left")

# Remove x and y ticks and labels to make the graph look neat
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel("")
ax.set_ylabel("")

plt.show()

# 4. Reading in the JACOBS storage tool data

The Jacobs NFM storage tool was developed to assess how much NFM storage is needed to reduce flood peaks in communities at risk. The tool uses the standard UK method for estimating flood flows (the Flood Estimation Handbook (FEH)) at 571 communities at Flood risk in the GMMC area. The tool calculates the volume of water that would be required to be stored upstream, to avoid flooding the downstream community at risk at different return period flows.

## Tool attributes

The underpinning dataset (Calculated_Hydrographs.csv) contains the following attributes:

- 'Unique_id': Specific flow location just upstream of a community
- 'SoP': standard of protection: indication of the lowest threshold of the risk for that community. Mising values == natural water courses, RP5 is default
- 'RP': the RP for a given scenario
- 'volume': proportion of hydrograph above the SoP threshold line, per timestep
- 'QT': Flow at a specified timestep in the design even. measure in cubic meters per second
- 'Flow': peak flow for a given RP


## Assumptions relating to the use of the Jacobs tool
- FEH method provides useful design hydrographs representative of the assigned return period.
- A baseline standard of protection of RP5.
- The properties at risk are at risk from river flooding based on the equivalent return period design storm from FEH.
- Communities at risk that are situated at the confluence of two rivers combine the design hydrographs of both branches into a single set of hydrographs for use in the calculations.

## Running the code

Nothing in this section should be changed. Similar to section 1, the code in this section only needs to be run once per session to read in the JACOBS storage data data. Reading data into memory is relatively slow, so it's better to avoid reading everything in again if you don't have to. The JACOBS dataset is the largest file that has to be read into this workbook. I have therefore created a faster-to-load version of the file called 'Calculated_Hydrographs.p' which won't be human readable. If for some reason the JACOBS stoarage dataset 'Calculated_Hydrographs.csv' is ever changed, you can recreate this .p file by running the second code block below. To run it, first remove the triple quote marks at the start and end of the code. Create the .p file and download it manually from the files sidebar. Replace the original .p file in the files to upload folder and use that in the future . Replace the triple quotes when you are done.

## Communities at Risk
See below for the table of communities at risk and which design storm IDs are associated with them. If there are two IDs this means that the CaR sits at the confluence of two rivers.


In [0]:
# Read in the communities at risk file names and associated Jacobs storage tool unique IDs
CaR_ID_df = pd.read_csv(f"{data_path}/CaR_name_ID.csv")

# Uploading the files to DBFS has replaced spaces and special characters with underscores, so when we
# read the IDs file in we need to match the new filenames. 
CaR_ID_df["property_report_name"] = (
    CaR_ID_df["property_report_name"]
    .str.replace("&", "_")  # Replace "&" with "_"
    .str.replace(" ", "_")  # Replace spaces with "_"
)

CaR_ID_df.head()
a = CaR_ID_df.jacobs_storage_tool_ID.values
CaR_IDs = []
for i in a:
  try:
    CaR_IDs.append(int(i))
  except:
    j = [int(k) for k in i.split("/")]
    CaR_IDs.extend(j)

CaR_IDs = list(set(CaR_IDs))
CaR_ID_df.loc[:, ["property_report_name", "jacobs_storage_tool_ID"]]

In [0]:
# This code block creates the .p file from the .csv file. To run it, first remove the triple quote marks at the start and end of the code. Create the .p file and download it. Replace them when you are done.
'''# Read in the Jacobs storage tool data (this stage is a bit slow because the file is large)
NFM_storage = gpd.read_file('/content/Calculated_Hydrographs.csv')

# Create a geometry column
NFM_storage['geometry'] = NFM_storage.apply(lambda row: Point(row['SnappedX'], row['SnappedY']), axis=1)

# Convert to GeoDataFrame and set CRS
NFM_storage = gpd.GeoDataFrame(NFM_storage, geometry='geometry')
NFM_storage.set_crs(epsg=27700, inplace=True)

columns_to_int = ['Unique_ID']
columns_to_float = ['Flow', 'timestep', 'QT']

# Convert each column to int
NFM_storage[columns_to_int] = NFM_storage[columns_to_int].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
NFM_storage[columns_to_float] = NFM_storage[columns_to_float].apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)

filehandler = open("Calculated_Hydrographs.p","wb")
pickle.dump(NFM_storage, filehandler)
filehandler.close()'''

In [0]:
# Read in Jacobs storage tool pre-processed data
file = open(f"{data_path}/Calculated_Hydrographs.p","rb")
NFM_storage = pickle.load(file)
file.close()

In [0]:
# Find unique communities at risk in the Jacobs storage tool
COR = NFM_storage.loc[(NFM_storage.RP == "RP2")&(NFM_storage.timestep == 0.1)]

In [0]:
# Join the Jacobs and JBA data
joined_gdf = gpd.sjoin(COR, NFM_features_gdf, how="inner", predicate="within")
irwell_WB_IDs = list(set(list(joined_gdf.loc[joined_gdf.Unique_ID.isin(CaR_IDs)].EA_WB_ID.values)))
irwell_WB_IDs

# Get the Upper Irwell River network
ui_gdf = gpd.read_file(f"{data_path}/ui_watercourse.shp")

# Plot with Geopandas, specifying the column and color map
ax = NFM_features_gdf.loc[NFM_features_gdf.EA_WB_ID.isin(irwell_WB_IDs)].plot(column="total_NFM_storage", cmap="viridis", alpha=0.75, legend=True, legend_kwds={"label":"Total NFM storage (cubic m)"}, zorder=1)
ui_gdf.plot(ax=ax, zorder=2)
joined_gdf.loc[joined_gdf.Unique_ID.isin(CaR_IDs)].plot(ax=ax, color="black", zorder=3)


# Remove x and y ticks and labels to make the graph neat.
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_title("Communities at risk in the Upper Irwell\nNFM layers selected: " + ", ".join(NFM_features_to_include))
plt.savefig(f"{results_path}/total_NFM_storage_and_CaR_map.png", dpi=800, bbox_inches="tight")
plt.show()

In [0]:
# Read in the FEH data to find community at risk catchment area
feh_data = pd.read_excel(f"{data_path}/FEH_Data.xlsx")

# 5. Functionality for calculating changes to communities at risk

In this section we define a function the combines all of the datasets together to find GIA and benefit/cost ratios for the catchments from the Defra funding calculator.

The DEFRA calculator is coded up as a seperate python module and imported as 'calculator'

This function uses the following method:
- Finds information related to the community of risk
- calculates the %area of the water body that is associated with the community at risk catchment
- Calculates the maximum potential NFM that could benefit that community
- Calculates the design floods for that community from the JACOBS data
- Calculates how much water needs to be stored to reduce the flood risk between categories
- Compares that volume to the maximum storage available
- Moves properties between flood risk categories according to the maximum reduction in flood risk available
- Calculates the Grant in Aid (GIA) available using the DEFRA calculator and returns other metrics of interest such as the Benefit Cost Ratio.

## Assumptions
- Flood risk in a community can be well described by the FEH design event
- NFM storage available is proportional to the fraction of the water body that the community's catchment area occupies.
- No sliding scale of changing flood risk, either all properties move flood risk category or they don't.
- The benefits calculation follows that currently used manually at the EA
- Costs are uniform per cubic meter of storage

## Running the code
This section of code is the main functionality of the workbook, however, it is not actually used until we call it with details of specific communities at risk. Nothing should be changed in this code block, but it does need to be executed once per session.

In [0]:
# Define the function that calculates GIA and other metrics of interest for each community
def calc_gia_per_community(community_name, cc_factor, storage_cost, duration_of_benefits, fraction_of_max_NFM=1):
  community_ids = [int(a) for a in CaR_ID_df.loc[CaR_ID_df.property_report_name == community_name].jacobs_storage_tool_ID.values[0].split("/")]
  community_X = int(joined_gdf.loc[joined_gdf.Unique_ID == community_ids[0]].SnappedX.values[0])
  community_Y = int(joined_gdf.loc[joined_gdf.Unique_ID == community_ids[0]].SnappedY.values[0])
  community_area = feh_data.loc[(feh_data.X == community_X)&(feh_data.Y == community_Y)].AREA.values[0]

  wfd_max_storage = joined_gdf.loc[joined_gdf.Unique_ID.isin(community_ids)].total_NFM_storage.sum() # this will be found from the JBA map
  wfd_area = joined_gdf.loc[joined_gdf.Unique_ID.isin(community_ids)].Area_km2.sum() #km2
    
  # Find the maximum NFM storage available to the community at risk
  community_max_storage = (community_area/wfd_area) * wfd_max_storage * fraction_of_max_NFM

  # Find the household data
  POR = pd.read_excel(f"{data_path}/{community_name}" + ".xlsx", sheet_name="RoFRS (excl. upper)")

  num_households_at_risk_today = pd.DataFrame({
      "deprivation_level": ["20% most deprived", "21% to 40% most deprived", "60% least deprived"],
      "low_risk": POR.iloc[1:4, 3].values,  # keep fixed (protected in worksheet)
      "moderate_risk": POR.iloc[1:4, 4].values,  # three entries, one per deprivation level
      "intermediate_risk": POR.iloc[1:4, 5].values,  # as above ...
      "significant_risk": POR.iloc[1:4, 6].values,
      "very_significant_risk": POR.iloc[1:4, 7].values,
  })

  num_households_at_risk_NFM = pd.DataFrame({
      "deprivation_level": ["20% most deprived", "21% to 40% most deprived", "60% least deprived"],
      "low_risk": POR.iloc[1:4, 3].values,  # keep fixed (protected in worksheet)
      "moderate_risk": [0, 0, 0],  # create an empty dataframe for calculating future risk
      "intermediate_risk": [0, 0, 0],  # as above ...
      "significant_risk": [0, 0, 0],
      "very_significant_risk": [0, 0, 0],
  })


  # Find the RP flows
  RP5 = NFM_storage.loc[(NFM_storage.Unique_ID.isin(community_ids))&(NFM_storage.RP == "RP5")&(NFM_storage.timestep == 0.1)].Flow.sum()
  RP20 = NFM_storage.loc[(NFM_storage.Unique_ID.isin(community_ids))&(NFM_storage.RP == "RP20")&(NFM_storage.timestep == 0.1)].Flow.sum()
  RP50 = NFM_storage.loc[(NFM_storage.Unique_ID.isin(community_ids))&(NFM_storage.RP == "RP50")&(NFM_storage.timestep == 0.1)].Flow.sum()
  RP100 = NFM_storage.loc[(NFM_storage.Unique_ID.isin(community_ids))&(NFM_storage.RP == "RP100")&(NFM_storage.timestep == 0.1)].Flow.sum()
  RP200 = NFM_storage.loc[(NFM_storage.Unique_ID.isin(community_ids))&(NFM_storage.RP == "RP200")&(NFM_storage.timestep == 0.1)].Flow.sum()

  # Find the flows

  RP20_hydrograph = NFM_storage.loc[(NFM_storage.Unique_ID.isin(community_ids))&(NFM_storage.RP == "RP20")].groupby("timestep")["QT"].sum().to_numpy()
  RP20_timesteps = np.sort(NFM_storage.loc[(NFM_storage.Unique_ID.isin(community_ids))&(NFM_storage.RP == "RP20")].timestep.unique())

  RP50_hydrograph = NFM_storage.loc[(NFM_storage.Unique_ID.isin(community_ids))&(NFM_storage.RP == "RP50")].groupby("timestep")["QT"].sum().to_numpy()
  RP50_timesteps = np.sort(NFM_storage.loc[(NFM_storage.Unique_ID.isin(community_ids))&(NFM_storage.RP == "RP50")].timestep.unique())

  RP100_hydrograph = NFM_storage.loc[(NFM_storage.Unique_ID.isin(community_ids))&(NFM_storage.RP == "RP100")].groupby("timestep")["QT"].sum().to_numpy()
  RP100_timesteps = np.sort(NFM_storage.loc[(NFM_storage.Unique_ID.isin(community_ids))&(NFM_storage.RP == "RP100")].timestep.unique())

  RP200_hydrograph = NFM_storage.loc[(NFM_storage.Unique_ID.isin(community_ids))&(NFM_storage.RP == "RP200")].groupby("timestep")["QT"].sum().to_numpy()
  RP200_timesteps = np.sort(NFM_storage.loc[(NFM_storage.Unique_ID.isin(community_ids))&(NFM_storage.RP == "RP200")].timestep.unique())
 

  # Identify changes to the hydrograph based on available storage for different return periods

  household_risk_changes = {}  # used to help with pv whole life benefit calculation

  #RP5
  RP5_20_vol = 0
  for q in RP20_hydrograph:
    if q > RP5:
      RP5_20_vol += (q-RP5)*(0.1*3600)

  RP5_50_vol = 0
  for q in RP50_hydrograph:
    if q > RP5:
      RP5_50_vol += (q-RP5)*(0.1*3600)

  RP5_100_vol = 0
  for q in RP100_hydrograph:
    if q > RP5:
      RP5_100_vol += (q-RP5)*(0.1*3600)

  RP5_200_vol = 0
  for q in RP200_hydrograph:
    if q > RP5:
      RP5_200_vol += (q-RP5)*(0.1*3600)


  #move RP5
  #print(community_max_storage)
  if community_max_storage > RP5_200_vol:
    #print("move Very Significant Risk to Low Risk")
    num_households_at_risk_NFM.low_risk += num_households_at_risk_today.very_significant_risk
    household_risk_changes[("very_significant_risk", "low_risk")] = num_households_at_risk_today.very_significant_risk.sum()
  elif community_max_storage > RP5_100_vol:
    #print("move Very Significant Risk to Moderate Risk")
    num_households_at_risk_NFM.moderate_risk += num_households_at_risk_today.very_significant_risk
    household_risk_changes[("very_significant_risk", "moderate_risk")] = num_households_at_risk_today.very_significant_risk.sum()
  elif community_max_storage > RP5_50_vol:
    #print("move Very Significant Risk to Intermediate Risk")
    num_households_at_risk_NFM.intermediate_risk += num_households_at_risk_today.very_significant_risk
    household_risk_changes[("very_significant_risk", "intermediate_risk")] = num_households_at_risk_today.very_significant_risk.sum()
  elif community_max_storage > RP5_20_vol:
    #print("move Very Significant Risk to Significant Risk")
    num_households_at_risk_NFM.significant_risk += num_households_at_risk_today.very_significant_risk
    household_risk_changes[("very_significant_risk", "significant_risk")] = num_households_at_risk_today.very_significant_risk.sum()
  else:
    num_households_at_risk_NFM.very_significant_risk += num_households_at_risk_today.very_significant_risk
    #print("No change (Very Significant Risk)")

  #RP20
  RP20_50_vol = 0
  for q in RP50_hydrograph:
    if q > RP20:
      RP20_50_vol += (q-RP20)*(0.1*3600)

  RP20_100_vol = 0
  for q in RP100_hydrograph:
    if q > RP20:
      RP20_100_vol += (q-RP20)*(0.1*3600)

  RP20_200_vol = 0
  for q in RP200_hydrograph:
    if q > RP20:
      RP20_200_vol += (q-RP20)*(0.1*3600)

  
    # move RP20
  #print(community_max_storage)
  if community_max_storage > RP20_200_vol:
    #print("move Significant Risk to Low Risk")
    num_households_at_risk_NFM.low_risk += num_households_at_risk_today.significant_risk
    household_risk_changes[("significant_risk", "low_risk")] = num_households_at_risk_today.significant_risk.sum()
  elif community_max_storage > RP20_100_vol:
    #print("move Significant Risk to Moderate Risk")
    num_households_at_risk_NFM.moderate_risk += num_households_at_risk_today.significant_risk
    household_risk_changes[("significant_risk", "moderate_risk")] = num_households_at_risk_today.significant_risk.sum()
  elif community_max_storage > RP20_50_vol:
    #print("move Significant Risk to Intermediate Risk")
    num_households_at_risk_NFM.intermediate_risk += num_households_at_risk_today.significant_risk
    household_risk_changes[("significant_risk", "intermediate_risk")] = num_households_at_risk_today.significant_risk.sum()
  else:
    num_households_at_risk_NFM.significant_risk += num_households_at_risk_today.significant_risk
    #print("No change (Significant Risk)")


  #RP50
  RP50_100_vol = 0
  for q in RP100_hydrograph:
    if q > RP50:
      RP50_100_vol += (q-RP50)*(0.1*3600)

  RP50_200_vol = 0
  for q in RP200_hydrograph:
    if q > RP50:
      RP50_200_vol += (q-RP50)*(0.1*3600)
  


  # move RP50
  #print(community_max_storage)
  if community_max_storage > RP50_200_vol:
    #print("move Intermediate Risk to Low Risk")
    num_households_at_risk_NFM.low_risk += num_households_at_risk_today.intermediate_risk
    household_risk_changes[("intermediate_risk", "low_risk")] = num_households_at_risk_today.intermediate_risk.sum()
  elif community_max_storage > RP50_100_vol:
    #print("move Intermediate Risk to Moderate Risk")
    num_households_at_risk_NFM.moderate_risk += num_households_at_risk_today.intermediate_risk
    household_risk_changes[("intermediate_risk", "moderate_risk")] = num_households_at_risk_today.intermediate_risk.sum()
  else:
    num_households_at_risk_NFM.intermediate_risk += num_households_at_risk_today.intermediate_risk
    #print("No change (Intermediate Risk)")

  #RP100

  RP100_200_vol = 0
  for q in RP200_hydrograph:
    if q > RP100:
      RP100_200_vol += (q-RP100)*(0.1*3600)
  


  # move RP100
  #print(community_max_storage)
  if community_max_storage > RP100_200_vol:
    #print("move Moderate Risk to Low Risk")
    num_households_at_risk_NFM.low_risk += num_households_at_risk_today.moderate_risk
    household_risk_changes[("moderate_risk", "low_risk")] = num_households_at_risk_today.moderate_risk.sum()
  else:
    num_households_at_risk_NFM.moderate_risk += num_households_at_risk_today.moderate_risk
    #print("No change (Moderate Risk)")

  pv_WLB_for_appraisal_period = calculator.calculate_pv_wlb(
      household_risk_changes,
      rofrs_damages_path=f"{data_path}/{community_name}" + ".xlsx",  # path to community workbook containing "RoFRS Damages" worksheet
      duration_of_benefits_DoB_period=duration_of_benefits
  )


  bcr, gia = calculator.calculate_gia(
    community_max_storage*storage_cost,
    pv_WLB_for_appraisal_period,
    num_households_at_risk_today,
    num_households_at_risk_NFM,
    num_households_at_risk_2040_5b=num_households_at_risk_today,
    num_households_at_risk_after_duration_of_benefits_5b=num_households_at_risk_NFM,
    duration_of_benefits_DoB_period=duration_of_benefits
    )


  return(np.around(bcr, 2),
         np.around(gia, 2),
         np.around(gia*100/(community_max_storage*storage_cost), 2),
         np.around(community_max_storage*storage_cost, 2),
         np.around(pv_WLB_for_appraisal_period, 2),
         num_households_at_risk_today,
         num_households_at_risk_NFM,
         RP5,
         RP20,
         RP50,
         RP100,
         RP200,
         RP20_timesteps,
         RP20_hydrograph,
         RP50_timesteps,
         RP50_hydrograph,
         RP100_timesteps,
         RP100_hydrograph,
         RP200_timesteps,
         RP200_hydrograph)


# 6. Calculating changes to an individual community

Now we start getting into the code you can change! The code cells in this section investigate changes in flood risk to a single community at risk. They call the function in section 5 and produce a series of tables and figures

## Running the code
You can change the following lines of code:



```
cc_factor = 1.26
community_name = 'Woolfold'
storage_cost = 20 # £ per cubic m
duration_of_benefits = 50

```

- cc_factor is the climate change factor you wish to explore. It is a simple multiple of the current flood risk. You can explore different uplift factors below. Note that it needs to be written as a scalar rather than a percentage (i.e an uplift of 26% is a scalar of 1.26)
- community_name is the name of the community you wish to investigate. Ensure that you use the quote marks and spell the name exactly as it is in the files
- storage_cost is the cost per cubic meter of storage.
- duration_of_benefits is the duration of benefits to be used in the DEFRA calculator. The calculations are sensitive to the duration of benefits- the longer the benefits period the higher the Benefit Cost Ratio. As we don’t have a figure for ongoing future costs, a rule of thumb is to use 50 years at an early stage, but refine it later. 

Simply edit the value after the equals sign and run the cells in this section


## Climate uplift factors

For reference (not currently used in the code) Irwell Management Catchment peak river flow allowances:

Period | Central |	Higher |	Upper
--- | --- |--- | ---
2020s|	12%	|15%	|24%
2050s	|19%	|26%	|43%
2080s	|35%|	46%|	75%

From https://environment.data.gov.uk/hydrology/climate-change-allowances/river-flow?mgmtcatid=3042

## Results
The cells below outputs:
- Values for the benefit cost ratio, GIA, costs and damages
- A static hydrograph of the design storms from FEH for the baseline and with the cc uplift factor applied. This image can be copy and pasted into a report.
- An interactive version of that graph if you wish to explore the data further
- A plot of return period peak flows with and without climate uplift factors
- A table of data showing households at risk today
- A table of data showing households at risk under the NFM scheme


In [0]:
# Investigate changes to properties at risk in a given community
cc_factor = 1.43
community_name = "Bacup_3"
storage_cost = 1 # £ per cubic m
duration_of_benefits = 50
fraction_of_max_NFM_to_implement=1

bcr, gia, pf_score, costs, damages, num_households_at_risk_today, num_households_at_risk_NFM, \
    RP5, RP20, RP50, RP100, RP200, RP20_timesteps, RP20_hydrograph, RP50_timesteps, \
    RP50_hydrograph, RP100_timesteps, RP100_hydrograph, RP200_timesteps, RP200_hydrograph = calc_gia_per_community(
        community_name, cc_factor, storage_cost, duration_of_benefits, fraction_of_max_NFM = fraction_of_max_NFM_to_implement
    )

print("Community at Risk:", community_name)
print("NFM layers selected: " + ", ".join(NFM_features_to_include))
print("Fraction of Max NFM to implement:", fraction_of_max_NFM_to_implement)
print("Benefit Cost Ratio:", bcr)
print("Partnership Funding Score:", pf_score, "%")
print("GIA:", gia)
print("Costs: £", costs)
print("Damages: £", damages)

In [0]:
# For a single community

# Plot the flows

plt.hlines(y=RP5, xmin=0.1, xmax=10.2, color="black", label="RP5")

plt.plot(RP20_timesteps, RP20_hydrograph, label="RP20", color="#66c2a5")
plt.plot(RP20_timesteps, RP20_hydrograph*cc_factor, label="RP20_CC", ls="--", color="#66c2a5")

plt.plot(RP50_timesteps, RP50_hydrograph, label="RP50", color="#fc8d62")
plt.plot(RP50_timesteps, RP50_hydrograph*cc_factor, label="RP50_CC", ls="--", color="#fc8d62")

plt.plot(RP100_timesteps, RP100_hydrograph, label="RP100", color="#8da0cb")
plt.plot(RP100_timesteps, RP100_hydrograph*cc_factor, label="RP100_CC", ls="--", color="#8da0cb")

plt.plot(RP200_timesteps, RP200_hydrograph, label="RP200", color="#e78ac3")
plt.plot(RP200_timesteps, RP200_hydrograph*cc_factor, label="RP200", ls="--", color="#e78ac3")

plt.legend()
plt.ylabel("Flow (cumecs)")
plt.xlabel("Time (hours)")
plt.title("Design flood events for " + community_name)
plt.savefig(f"{results_path}/Design_flood_events_for_" + community_name + ".png", dpi=800, bbox_inches="tight")


In [0]:
# Create figure
fig = go.Figure()

# Add RP5 horizontal line
fig.add_trace(go.Scatter(
    x=[min(RP20_timesteps), max(RP20_timesteps)],
    y=[RP5, RP5],
    mode="lines",
    line=dict(color="black", width=2),
    name="RP5",
    hoverinfo="skip"  # No need for hover on static line
))

# Define colours
colors = {
    "RP20": "#66c2a5",
    "RP50": "#fc8d62",
    "RP100": "#8da0cb",
    "RP200": "#e78ac3"
}

# Add hydrographs (original + climate change versions)
for rp, timesteps, hydrograph, color in zip(
    ["RP20", "RP50", "RP100", "RP200"],
    [RP20_timesteps, RP50_timesteps, RP100_timesteps, RP200_timesteps],
    [RP20_hydrograph, RP50_hydrograph, RP100_hydrograph, RP200_hydrograph],
    colors.values()
):
    # Base hydrograph
    fig.add_trace(go.Scatter(
        x=timesteps,
        y=hydrograph,
        mode="lines",
        line=dict(color=color, width=2),
        name=rp,
        hovertemplate=(
            "Time (hours): %{x}<br>" +
            "Flow (cumecs): %{y}<br>" +
            "Return Period: " + rp
        )
    ))

    # Climate change hydrograph (dashed)
    fig.add_trace(go.Scatter(
        x=timesteps,
        y=hydrograph * cc_factor,
        mode="lines",
        line=dict(color=color, width=2, dash="dash"),
        name=f"{rp}_CC",
        hovertemplate=(
            "Time (hours): %{x}<br>" +
            "Flow (cumecs): %{y}<br>" +
            "Return Period: " + f"{rp}_CC"
        )
    ))

# Customise layout
fig.update_layout(
    title=f"Design flood events for {community_name}",
    xaxis_title="Time (hours)",
    yaxis_title="Flow (cumecs)",
    legend_title="Return Periods",
    template="plotly_white"
)

# Show plot
fig.show()

In [0]:
plt.plot([5, 20, 50, 100, 200], [RP5, RP20, RP50, RP100, RP200], label="baseline period")
plt.plot([5, 20, 50, 100, 200], np.array([RP5, RP20, RP50, RP100, RP200])*cc_factor, label="2040")
plt.legend()
plt.xlabel("Return Period")
plt.ylabel("Flow (cumecs)")
plt.title("RP Flows for " + community_name)
plt.savefig(f"{results_path}/Return_period_flows_for_" + community_name + ".png", dpi=800, bbox_inches="tight")

plt.show()

In [0]:
# Show the number of households at risk today

print("Number of households at risk today in " + community_name)
num_households_at_risk_today

In [0]:
# Show the number of households at risk under maximum NFM storage
print("Number of households at risk under maximum NFM scenario in " + community_name)
print("NFM layers selected: " + ", ".join(NFM_features_to_include))
num_households_at_risk_NFM

In [0]:
# File name- inculdes the datetime stamp so that your files don"t get overwritten
file_name = f"{results_path}/households_risk_report" + community_name + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")+ ".csv"

# Writing to CSV
with open(file_name, "w") as f:
    f.write("Community at Risk:" + community_name + "\n")
    f.write("NFM layers selected: " + ", ".join(NFM_features_to_include) + "\n")
    f.write("Fraction of Max NFM to implement: " + str(fraction_of_max_NFM_to_implement) + "\n")
    f.write("Climate Change Factor: " + str(cc_factor) + "\n")
    f.write("Storage cost: " +  str(storage_cost) + "£ per cubic m" + "\n")
    f.write("Duration of benefits: " + str(duration_of_benefits) + "\n")
    f.write("Benefit Cost Ratio: " + str(bcr) + "\n")
    f.write("Partnership Funding Score: " + str(pf_score) + "%" + "\n")
    f.write("GIA: " + str(gia) + "\n")
    f.write("Costs: £" + str(costs) + "\n")
    f.write("Damages: £" + str(damages) + "\n")
    f.write("\n")
    f.write(f"Number of households at risk today in {community_name}\n")
num_households_at_risk_today.to_csv(file_name, mode="a", index=False)

with open(file_name, "a") as f:
    f.write(f"\nNumber of households at risk under maximum NFM scenario in {community_name}\n")
    f.write(f"NFM layers selected: {', '.join(NFM_features_to_include)}\n")
num_households_at_risk_NFM.to_csv(file_name, mode="a", index=False)
f.close()

In [0]:

for community_name in CaR_ID_df.property_report_name.values:
    
    # Investigate changes to properties at risk in a given community
    cc_factor = 1.43
    storage_cost = 1 # £ per cubic m
    duration_of_benefits = 50
    fraction_of_max_NFM_to_implement=1
    
    bcr, gia, pf_score, costs, damages, num_households_at_risk_today, num_households_at_risk_NFM, \
        RP5, RP20, RP50, RP100, RP200, RP20_timesteps, RP20_hydrograph, RP50_timesteps, \
        RP50_hydrograph, RP100_timesteps, RP100_hydrograph, RP200_timesteps, RP200_hydrograph = calc_gia_per_community(
            community_name, cc_factor, storage_cost, duration_of_benefits, fraction_of_max_NFM = fraction_of_max_NFM_to_implement
        )
    
    
    # Plot the flows
    
    plt.hlines(y=RP5, xmin=0.1, xmax=10.2, color="black", label="RP5")
    
    plt.plot(RP20_timesteps, RP20_hydrograph, label="RP20", color="#66c2a5")
    plt.plot(RP20_timesteps, RP20_hydrograph*cc_factor, label="RP20_CC", ls="--", color="#66c2a5")
    
    plt.plot(RP50_timesteps, RP50_hydrograph, label="RP50", color="#fc8d62")
    plt.plot(RP50_timesteps, RP50_hydrograph*cc_factor, label="RP50_CC", ls="--", color="#fc8d62")
    
    plt.plot(RP100_timesteps, RP100_hydrograph, label="RP100", color="#8da0cb")
    plt.plot(RP100_timesteps, RP100_hydrograph*cc_factor, label="RP100_CC", ls="--", color="#8da0cb")
    
    plt.plot(RP200_timesteps, RP200_hydrograph, label="RP200", color="#e78ac3")
    plt.plot(RP200_timesteps, RP200_hydrograph*cc_factor, label="RP200", ls="--", color="#e78ac3")
    
    plt.legend()
    plt.ylabel("Flow (cumecs)")
    plt.xlabel("Time (hours)")
    plt.title("Design flood events for " + community_name)
    plt.savefig(f"{results_path}/Design_flood_events_for_" + community_name + ".png", dpi=800, bbox_inches="tight")
    
    plt.clf()
    
    plt.plot([5, 20, 50, 100, 200], [RP5, RP20, RP50, RP100, RP200], label="baseline period")
    plt.plot([5, 20, 50, 100, 200], np.array([RP5, RP20, RP50, RP100, RP200])*cc_factor, label="2040")
    plt.legend()
    plt.xlabel("Return Period")
    plt.ylabel("Flow (cumecs)")
    plt.title("RP Flows for " + community_name)
    plt.savefig(f"{results_path}/Return_period_flows_for_" + community_name + ".png", dpi=800, bbox_inches="tight")
    plt.clf()
    # File name
    file_name = f"{results_path}/households_risk_report" + community_name + ".csv"
    
    # Writing to CSV
    with open(file_name, "w") as f:
        f.write("Community at Risk:" + community_name + "\n")
        f.write("NFM layers selected: " + ", ".join(NFM_features_to_include) + "\n")
        f.write("Fraction of Max NFM to implement: " + str(fraction_of_max_NFM_to_implement) + "\n")
        f.write("Climate Change Factor: " + str(cc_factor) + "\n")
        f.write("Storage cost: " +  str(storage_cost) + "£ per cubic m" + "\n")
        f.write("Duration of benefits: " + str(duration_of_benefits) + "\n")
        f.write("Benefit Cost Ratio: " + str(bcr) + "\n")
        f.write("Partnership Funding Score: " + str(pf_score) + "%" + "\n")
        f.write("GIA: " + str(gia) + "\n")
        f.write("Costs: £" + str(costs) + "\n")
        f.write("Damages: £" + str(damages) + "\n")
        f.write("\n")
        f.write(f"Number of households at risk today in {community_name}\n")
    num_households_at_risk_today.to_csv(file_name, mode="a", index=False)
    
    with open(file_name, "a") as f:
        f.write(f"\nNumber of households at risk under maximum NFM scenario in {community_name}\n")
        f.write(f"NFM layers selected: {', '.join(NFM_features_to_include)}\n")
    num_households_at_risk_NFM.to_csv(file_name, mode="a", index=False)
    f.close()


## Single community optimisation of NFM

We can also investigate changing the fraction of the maximum available NFM to be implemented in a project to see if we can achieve better cost benefit ratios. 

Here we have set up the same code as before, but now we increment the fraction of the maximum available NFM to use to see if we can get a better cost benefit ratio using less NFM. 

This code block can run very slowly if we ask it to try lots of different fractions. If we ask it to simulate 100 increments (1% of max NFM, 2% of max NFM, 3% of max NFM etc. up to 100% max NFM) that will take ten times as long as checking 10 increments (10% of max NFM, 20% max NFM, 30% of max NFM etc.) You can change the line of code that controls this. Use:

```fractions = np.linspace(0.01, 1, 100)``` for 100 increments or

```fractions = np.linspace(0.1, 1, 10)``` for 10 increments

The plot below shows how the benefit cost ratio changes with % max NFM implemented.


In [0]:
# Investigate changes to properties at risk in a given community
cc_factor = 1.43
community_name = "Bacup_3"
storage_cost = 1 # £ per cubic m
duration_of_benefits = 50


# fractions = np.linspace(0.01, 1, 100) # Use these values if you want to have a very high resolution optimisation. This will try 100 different NFM fractions
fractions = np.linspace(0.1, 1, 10) # This will try 10 different NFM fractions
bcr_list = []

for f in fractions:
    bcr, gia, pf_score, costs, damages, num_households_at_risk_today, num_households_at_risk_NFM, \
        RP5, RP20, RP50, RP100, RP200, RP20_timesteps, RP20_hydrograph, RP50_timesteps, \
        RP50_hydrograph, RP100_timesteps, RP100_hydrograph, RP200_timesteps, RP200_hydrograph = calc_gia_per_community(
            community_name, cc_factor, storage_cost, duration_of_benefits, fraction_of_max_NFM=f)

    bcr_list.append(bcr)

optimise_catch_df = pd.DataFrame({"Fraction of Max NFM": fractions, "Benefit cost ratio":bcr_list})

filename = f"{results_path}/optimised_NFM_fraction_for_" + community_name + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")+ ".csv"

# Construct the header text
header_text = (
    f"Incremental implementation of NFM in {community_name}\n"
    f"NFM layers selected: {', '.join(NFM_features_to_include)}\n"
    f"Storage cost: £{storage_cost} per cubic m\n"
    f"Duration of benefits: {duration_of_benefits} years\n"
    f"Maximum NFM storage available: {np.around(costs / storage_cost)} cubic m\n"
)

# Write header to file
with open(filename, "w") as f:
    f.write(header_text + "\n")  # Write header and add a blank line

# Append DataFrame to the same file
optimise_catch_df.to_csv(filename, mode="a", index=False, float_format="%.2f")


In [0]:
plt.plot(fractions, bcr_list)
plt.xlabel("fraction of max NFM used")
plt.ylabel("Cost benefit Ratio")
plt.title(header_text)
plt.savefig(f"{results_path}/optimised_NFM_fraction_for_" + community_name + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")+ ".png", dpi=800, bbox_inches="tight")


# 7. Calculating funding information for the whole of the Upper Irwell

This section calculates the same information for every community in the Upper Irwell and provides summary maps and tables of data rather than individual community hydrographs and property information.

## Running the code
You can change the following lines of code:

```
cc_factor = 1.26
storage_cost = 20 # £ per cubic m
duration_of_benefits = 50

```

- cc_factor is the climate change factor you wish to explore. It is a simple multiple of the current flood risk. You can explore different uplift factors below. Note that it needs to be written as a scalar rather than a percentage (i.e an uplift of 26% is a scalar of 1.26)
- storage_cost is the cost per cubic meter of storage.
- duration_of_benefits is the duration of benefits to be used in the DEFRA calculator.

Simply edit the value after the equals sign and run the cells in this section

## Results
This section produces:
- A static map of communities coloured by the benefit cost ratio
- A table of communities prioritied by benefit cost ratio, sorted from highest BCR to lowest.
- An interactive map of communities coloured by the benefit cost ratio


In [0]:
# For multiple communities
# Lines to change
cc_factor = 1.26
storage_cost = 1 # £ per cubic m
duration_of_benefits = 50

# Create a dataframe to store the results in
prioritisation_df = pd.DataFrame({"community_name":[], "Unique_ID":[], "gia":[], "bcr":[], "PF_score":[], "costs":[], "damages":[]})
ticker = 0

#Loop through every community in the Upper Irwell
for community_name in CaR_ID_df.property_report_name.values:
  bcr, gia, pf_score, costs, damages, num_households_at_risk_today, num_households_at_risk_NFM, RP5, RP20, RP50, RP100, RP200, RP20_timesteps, RP20_hydrograph, RP50_timesteps, RP50_hydrograph, RP100_timesteps, RP100_hydrograph, RP200_timesteps, RP200_hydrograph = calc_gia_per_community(community_name, cc_factor, storage_cost, duration_of_benefits)
  community_id = int(CaR_ID_df.loc[CaR_ID_df.property_report_name == community_name].jacobs_storage_tool_ID.values[0].split("/")[0])
  prioritisation_df.loc[ticker] = [community_name, community_id, gia, bcr, pf_score, costs, damages]
  ticker += 1


In [0]:
# Join the Jacobs and JBA data

merged_gdf = pd.merge(joined_gdf, prioritisation_df, on="Unique_ID", how="inner")
irwell_WB_IDs = list(set(list(joined_gdf.loc[joined_gdf.Unique_ID.isin(CaR_IDs)].EA_WB_ID.values)))
irwell_WB_IDs

# Plot with Geopandas, specifying the column and color map
ax = NFM_features_gdf.loc[NFM_features_gdf.EA_WB_ID.isin(irwell_WB_IDs)].plot(column="total_NFM_storage", cmap="viridis", alpha=0.75, legend=True, legend_kwds={"label":"Total NFM storage (cubic m)"})
merged_gdf.loc[merged_gdf.Unique_ID.isin(CaR_IDs)].plot(
    ax=ax,
    column="bcr",
    cmap="PiYG",  # Choose a suitable color map
    markersize=10,
    legend=True,
    legend_kwds={"label": "Benefit Cost Ratio"}
)

header_text = (
    f"Communities at risk in the Upper Irwell\n"
    f"NFM layers selected: {', '.join(map(str, NFM_features_to_include))}\n"
    f"Storage cost: £{storage_cost} per cubic m\n"
    f"Duration of benefits: {duration_of_benefits} years\n")
# Remove x and y ticks and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_title(header_text)
plt.savefig(f"{results_path}/BCR_map" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".png", dpi=800, bbox_inches="tight")
plt.show()

In [0]:
# Show the prioritised list of communities
# If you wish to see all of the rows, remove .head(20) from the line of code below.
print("Communities at risk in the Upper Irwell\nNFM layers selected: " + ", ".join(NFM_features_to_include))
prioritisation_df.sort_values(by='bcr', ascending=False)


In [0]:
# File name
file_name = f"{results_path}/prioritised_CaR" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".csv"

# Writing to CSV
with open(file_name, "w") as f:
    f.write(header_text)
prioritisation_df.sort_values(by="bcr", ascending=False).to_csv(file_name, mode="a", index=False)


f.close()


In [0]:
# Reproject the data to WGS84 (EPSG:4326)
gdf_wgs84 = NFM_features_gdf.to_crs(epsg=4326)
merged_gdf_wgs84 = merged_gdf.to_crs(epsg=4326)

# Create base map centered on reprojected data
m = folium.Map(
    location=[gdf_wgs84.geometry.centroid.y.mean(), gdf_wgs84.geometry.centroid.x.mean()],
    zoom_start=10,
    tiles='OpenStreetMap'
)

# Create colour maps
storage_colourmap = cm.LinearColormap(
    colors=[mpl_colors.rgb2hex(mpl_cm.viridis(i)) for i in range(256)],  # Use Viridis colourmap
    vmin=gdf_wgs84['total_NFM_storage'].min(),
    vmax=gdf_wgs84['total_NFM_storage'].max(),
    caption='Total NFM Storage'
)

# Add polygons (storage data)
folium.GeoJson(
    gdf_wgs84,
    style_function=lambda x: {
        'fillColor': storage_colourmap(x['properties']['total_NFM_storage']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.75
    },
    tooltip=folium.GeoJsonTooltip(fields=['total_NFM_storage'], aliases=['Storage: '])
).add_to(m)

# Normalise the bcr values for marker sizing
bcr_min = merged_gdf_wgs84['bcr'].min()
bcr_max = merged_gdf_wgs84['bcr'].max()

# Define marker size scaling function
def scale_bcr(value, min_size=3, max_size=25):
    """Scale the bcr values to a range of marker sizes."""
    return min_size + (value - bcr_min) / (bcr_max - bcr_min) * (max_size - min_size)

# Add markers with proportional sizes
for _, row in merged_gdf_wgs84.iterrows():
    size = scale_bcr(row['bcr'])  # Scale the marker size based on bcr
    folium.CircleMarker(
        location=[row.geometry.centroid.y, row.geometry.centroid.x],
        radius=size,  # Set radius based on scaled bcr
        color='#000000',
        fill=True,
        fill_color='#252525',
        fill_opacity=0.8,
        popup=f"BCR: {row['bcr']}<br>Community: {row['community_name']}"
    ).add_to(m)

# Add polygon colourmap to map
storage_colourmap.add_to(m)

# Display map
m


## Prioritisation with optimisation

Here we use the optimal fraction of NFM for each community at risk and rank the potential projects as above to see if we can achieve better cost benefit ratios. 

Here we have set up the same code as before, but now we increment the fraction of the maximum available NFM to use to see if we can get a better cost benefit ratio using less NFM. 

This code block can run very slowly as we are asking it to try lots of different fractions for EVERY CaR. If we ask it to simulate 100 increments (1% of max NFM, 2% of max NFM, 3% of max NFM etc. up to 100% max NFM) that will take ten times as long as checking 10 increments (10% of max NFM, 20% max NFM, 30% of max NFM etc.) You can change the line of code that controls this. Use:

```fractions = np.linspace(0.01, 1, 100)``` for 100 increments or

```fractions = np.linspace(0.1, 1, 10)``` for 10 increments

The table below shows prioritisation outputs for both the optimal fraction of max NFM implemented and the max NFM implemented. The map shows the prioiritsation as above but for the optimal CBR.


In [0]:
# For multiple communities
# Lines to change
cc_factor = 1.26
storage_cost = 1 # £ per cubic m
duration_of_benefits = 50

header_text = (
    f"Communities at risk in the Upper Irwell, optimised NFM fraction\n"
    f"NFM layers selected: {', '.join(NFM_features_to_include)}\n"
    f"Storage cost: £{storage_cost} per cubic m\n"
    f"Duration of benefits: {duration_of_benefits} years\n")

# Create a dataframe to store the results in
prioritisation_df = pd.DataFrame({"community_name":[], "Unique_ID":[], "Optimal_NFM_Fraction":[], "gia_optimal":[], "bcr_optimal":[], "PF_score_optimal":[], "costs_optimal":[], "damages_optimal":[], "gia_max":[], "bcr_max":[], "PF_score_max":[], "costs_max":[], "damages_max":[]})
ticker = 0

#Loop through every community in the Upper Irwell
for community_name in CaR_ID_df.property_report_name.values:
  #fractions = np.linspace(0.05, 1, 20)
  fractions = np.linspace(0.1, 1, 10)
  gia_list = []
  bcr_list = []
  pf_list = []
  costs_list = []
  damages_list = []

  for f in fractions:
    bcr, gia, pf_score, costs, damages, num_households_at_risk_today, num_households_at_risk_NFM, RP5, RP20, RP50, RP100, RP200, RP20_timesteps, RP20_hydrograph, RP50_timesteps, RP50_hydrograph, RP100_timesteps, RP100_hydrograph, RP200_timesteps, RP200_hydrograph = calc_gia_per_community(community_name, cc_factor, storage_cost, duration_of_benefits, fraction_of_max_NFM=f)
    community_id = int(CaR_ID_df.loc[CaR_ID_df.property_report_name == community_name].jacobs_storage_tool_ID.values[0].split("/")[0])
    gia_list.append(gia)
    bcr_list.append(bcr)
    pf_list.append(pf_score)
    costs_list.append(costs)
    damages_list.append(damages)
      
  i = np.argmax(np.array(bcr_list))
  prioritisation_df.loc[ticker] = [community_name, community_id, fractions[i], gia_list[i], bcr_list[i], pf_list[i], costs_list[i], damages_list[i], gia_list[-1], bcr_list[-1], pf_list[-1], costs_list[-1], damages_list[-1]]
  ticker += 1


In [0]:
print(header_text)
prioritisation_df.sort_values(by="bcr_optimal", ascending=False)

In [0]:
# File name
file_name = f"{results_path}/Optimised_prioritised_CaR.csv"

# Writing to CSV
with open(file_name, "w") as f:
    f.write(header_text)
prioritisation_df.sort_values(by="bcr_optimal", ascending=False).to_csv(file_name, mode="a", index=False)

f.close()

In [0]:
# Join the Jacobs and JBA data
merged_gdf = pd.merge(joined_gdf, prioritisation_df, on="Unique_ID", how="inner")
irwell_WB_IDs = list(set(list(joined_gdf.loc[joined_gdf.Unique_ID.isin(CaR_IDs)].EA_WB_ID.values)))
irwell_WB_IDs

# Plot with Geopandas, specifying the column and color map
ax = NFM_features_gdf.loc[NFM_features_gdf.EA_WB_ID.isin(irwell_WB_IDs)].plot(column="total_NFM_storage", cmap="viridis", alpha=0.75, legend=True, legend_kwds={"label":"Total NFM storage (cubic m)"})
merged_gdf.loc[merged_gdf.Unique_ID.isin(CaR_IDs)].plot(
    ax=ax,
    column="bcr_optimal",
    cmap="PiYG",  # Choose a suitable color map
    markersize=10,
    legend=True,
    legend_kwds={"label": "Benefit Cost Ratio"}
)

# Remove x and y ticks and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_title(header_text)
plt.savefig(f"{results_path}/BCR_optimised_map" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".png", dpi=800, bbox_inches="tight")
plt.show()